In [1]:
!pip install --upgrade pip
!pip install tensorflow 
!pip install openpyxl

In [2]:
import os

os.chdir("./content")

lm_dir = "tf-levenberg-marquardt"
if not os.path.exists(lm_dir):
  !git clone https://github.com/fabiodimarco/$lm_dir

os.chdir(lm_dir)

In [3]:
import numpy as np
import pandas as pd

def create_dataframe(output, label):
    df = pd.read_excel("../../../data.xlsx")
    x1, x2, x3, x4, x5  = df['x1'], df['x2'], df['x3'], df['x4'], df['x5']

    input = np.vstack([x1, x2, x3, x4, x5 ]).T
    output = np.array(df[output])
    return output, input

output, input = create_dataframe(output="y1", label="ph")

/tmp/ipykernel_97805/4275043310.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
import tensorflow as tf
import numpy as np
from keras import regularizers
from keras import initializers
import levenberg_marquardt as lm

# layers, neurons
class ShuffleArchitecture:
    def __init__(self, input_size, hidden_sizes, output_size, act_h, act_o, param_reg):
        self.input_size = input_size
        self.hidden_sizes = hidden_sizes
        self.output_size = output_size
        self.act_h = act_h
        self.act_o = act_o
        self.regularizer = regularizers.L2(param_reg)
        self.initializer = initializers.RandomUniform(minval=-0.5, maxval=0.5, seed=np.random.randint(1, 10000))

    def set_architecture(self):
        self.model = tf.keras.Sequential()
        self.model.add(tf.keras.layers.Dense(self.hidden_sizes[0],
                        input_shape=(self.input_size,),
                        activation=self.act_h,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,                        
                        ))  # input layer
        
        for size in self.hidden_sizes[1:]:  # hidden layers
            self.model.add(tf.keras.layers.Dense(size,
                            activation=self.act_h,
                            kernel_regularizer=self.regularizer,
                            kernel_initializer=self.initializer,  
                        ))

        self.model.add(tf.keras.layers.Dense(self.output_size,
                        activation=self.act_o,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,  
                        ))  # output layer

    def create_model(self, _learning_rate):
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=_learning_rate),
            loss=tf.keras.losses.MeanSquaredError())

        self.lm_model = lm.ModelWrapper(
            tf.keras.models.clone_model(self.model))

        self.lm_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=_learning_rate),
            loss=lm.MeanSquaredError())
        return(self.lm_model)

2024-03-24 20:51:22.686126: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-24 20:51:22.767561: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-24 20:51:22.767626: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-24 20:51:22.769127: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-24 20:51:22.779079: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-24 20:51:22.780251: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [5]:
import tensorflow as tf
import matplotlib.pyplot as plt
import time
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score, mean_squared_error


class TrainWithSmallDataset:
    def __init__(self, batch_size=1000):
        self.batch_size = batch_size
        self.betters = []
    
    def create_dataset(self, input, output):
      input = tf.expand_dims(tf.cast(input, tf.float32), axis=-1)
      output = tf.expand_dims(tf.cast(output, tf.float32), axis=-1)

      dataset = tf.data.Dataset.from_tensor_slices((input, output))
      dataset = dataset.shuffle(len(input))
      dataset = dataset.batch(self.batch_size).cache()
      dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
      return (dataset, input, output)

    def split_dataset(self, input, output):
      input_train, input_vt, output_train, output_vt = train_test_split(input, output, test_size=0.3, shuffle = True)
      input_val, input_test, output_val, output_test = train_test_split(input_vt, output_vt, test_size=0.5, shuffle = True)

      self.train_dataset, self.train_input, self.train_output = self.create_dataset(input_train, output_train)
      self.val_dataset, self.val_input, self.val_output = self.create_dataset(input_val, output_val)
      self.test_dataset, self.test_input, self.test_output = self.create_dataset(input_test, output_test)
      self.vt_dataset, self.vt_input, self.vt_output = self.create_dataset(input_vt, output_vt)
      self.dataset, self.input, self.output = self.create_dataset(input, output)

      self._train = (input_train, output_train)
      self._vt = (input_vt, output_vt)
      self._val = (input_val, output_val)
      self._test = (input_test, output_test)
 
 
    
    def train_using_lm(self, train_dataset, epochs=1000):
      early_stopping_monitor = EarlyStopping(monitor='val_loss',
                                              patience=6,
                                              restore_best_weights=True)

      t2_start = time.perf_counter()
      self.results = self.lm_model.fit(train_dataset,
                                            epochs=epochs,
                                            validation_data=self.val_dataset,
                                            callbacks=[early_stopping_monitor],
                                            verbose=0)
      t2_stop = time.perf_counter()
      print("Elapsed time: ", t2_stop - t2_start)
      print ("Stopped at epoch: ", early_stopping_monitor.stopped_epoch)

    def get_metrics(self):
          self.test_prediction = self.lm_model.predict(self.test_input)

          pred = self.lm_model.predict(self.input).flatten()
          test_pred = self.test_prediction.flatten()
          val_pred = self.lm_model.predict(self.val_input).flatten()
          vt_pred = self.lm_model.predict(self.vt_input).flatten()

          r2 = r2_score(self.output, pred)
          r2_test = r2_score(self.test_output, test_pred)
          r2_val = r2_score(self.val_output, val_pred)
          r2_vt = r2_score(self.vt_output, vt_pred)

          mse = mean_squared_error(self.output, pred)
          mse_test = mean_squared_error(self.test_output, test_pred)
          mse_val = mean_squared_error(self.val_output, val_pred)
          mse_vt = mean_squared_error(self.vt_output, vt_pred)

          metrics = {
                          'r2': r2,
                          'r2_test': r2_test,
                          'r2_val': r2_val,
                          'r2_vt': r2_vt,
                          'mse': mse,
                          'mse_test': mse_test,
                          'mse_val': mse_val,
                          'mse_vt': mse_vt
                          }

          return metrics

In [6]:
from itertools import product
import pickle

class Tester:
  def __init__(self, input, output, run_times=500, dataset_run_times=10):
    self.run_times = run_times
    self.better_metrics = {}
    self.dataset_run_times = dataset_run_times
    self.input, self.output = input, output
  
  def setArchitecure(self, trainer, _hidden_sizes, _pg, _lr):
    shuffler = ShuffleArchitecture(input_size=5,
                                    hidden_sizes=_hidden_sizes,
                                    output_size=1,
                                    act_h='tanh',
                                    act_o='linear',
                                    param_reg=_pg)
    shuffler.set_architecture()    
    trainer.lm_model = shuffler.create_model(_lr)

  def Train(self, trainer, epochs=1000):
    trainer.train_using_lm(trainer.train_dataset, epochs=epochs)
    return(trainer.get_metrics(), trainer.lm_model)

  def SaveModelWeights(self, model, fileName):
    path = f"../models/{fileName}.keras"
    open(path,'w').close()
    model.save_weights(path) 

  def SaveDataset(self, trainer, fileName):
    path = f"../dataset/{fileName}.pkl" 
    with open(path, 'wb') as f:
      pickle.dump((trainer._train, trainer._vt, trainer._val, trainer._test), f)


  def LoopWeights(self, sort_by, boundarie, trainer, idx):
    better_model = 0
    save = False

    for i in range(self.run_times):
      print (f"+++++++++++ [{idx}] | {i + 1} ++++++++++++++++++")
      metrics, model = self.Train(trainer)
      if (metrics[sort_by] >= boundarie): # should be <= to descending metrics
        fileName = f"model_{idx}_{better_model}"
        self.SaveModelWeights(model, fileName)
        self.better_metrics[fileName] = metrics
        better_model += 1
        save = True
    
    return(save)

# Init
  def Loop(self, sort_by, boundarie, hidden_sizes, regularizers, learning_rate):
    trainer = TrainWithSmallDataset()

    for count, (hidden_size, reg, lr) in enumerate(product(hidden_sizes, regularizers, learning_rate), start=1):
      header =  f"Hidden Size={hidden_size}, regularizer={reg}, learning_rate={lr}"
      print(f"Testando combinacao{count}: {header}")
      self.setArchitecure(trainer, hidden_size, reg, lr)
      for j in range(self.dataset_run_times):
        trainer.split_dataset(self.input, self.output)
        if (self.LoopWeights(sort_by, boundarie, trainer, f"{count}_{j}") == True):
          self.SaveDataset(trainer, f"dataset_{count}_{j}")
          self.DisplayBetterResults(sort_by, header, f"{count}_{j}")
        self.better_metrics = {}

  def DisplayBetterResults(self, sort_by, header, dataset=0):
    df = pd.DataFrame.from_dict(self.better_metrics, orient='index')
    df = df.sort_values([sort_by])
    display(df)
    path = f'../results/metrics_{dataset}'
    df.to_excel(f"{path}.xlsx", index=True)
    print(f"DataFrame salvo em {path}")
    with open(f"{path}.txt", 'w') as arquivo:
      arquivo.write(header)

In [7]:
tester = Tester(input, output, run_times=25, dataset_run_times=10)

In [8]:
tester.Loop(sort_by='r2',
            boundarie=0.8,
            hidden_sizes = [[20, 10], [30, 15], [40, 20], [45, 22]],
            regularizers=[0.01, 0.1],
            learning_rate=[0.1])

Testando combinacao1: Hidden Size=[20, 10], regularizer=0.01, learning_rate=0.1
+++++++++++ [1_0] | 1 ++++++++++++++++++
Elapsed time:  2.1487809890002154
Stopped at epoch:  27
1/1 [==============================] - 0s 21ms/step
+++++++++++ [1_0] | 2 ++++++++++++++++++
Elapsed time:  0.5584065990001363
Stopped at epoch:  7
1/1 [==============================] - 0s 23ms/step
+++++++++++ [1_0] | 3 ++++++++++++++++++
Elapsed time:  0.471062646000064
Stopped at epoch:  6
1/1 [==============================] - 0s 22ms/step
+++++++++++ [1_0] | 4 ++++++++++++++++++
Elapsed time:  0.9433533609999358
Stopped at epoch:  16
1/1 [==============================] - 0s 18ms/step
+++++++++++ [1_0] | 5 ++++++++++++++++++
Elapsed time:  0.6616098899999088
Stopped at epoch:  6
1/1 [==============================] - 0s 23ms/step
+++++++++++ [1_0] | 6 ++++++++++++++++++
Elapsed time:  0.40635887899998124
Stopped at epoch:  6
1/1 [==============================] - 0s 26ms/step
+++++++++++ [1_0] | 7 ++++++++

2024-03-24 20:52:24.415841: W tensorflow/core/data/root_dataset.cc:342] Optimization loop failed: CANCELLED: Operation was cancelled


Elapsed time:  2.427581336000003
Stopped at epoch:  31
1/1 [==============================] - 0s 24ms/step
+++++++++++ [1_1] | 10 ++++++++++++++++++
Elapsed time:  0.8372179579998829
Stopped at epoch:  8
1/1 [==============================] - 0s 20ms/step
+++++++++++ [1_1] | 11 ++++++++++++++++++
Elapsed time:  0.49609131000011075
Stopped at epoch:  7
1/1 [==============================] - 0s 26ms/step
+++++++++++ [1_1] | 12 ++++++++++++++++++
Elapsed time:  0.490910397999869
Stopped at epoch:  0
1/1 [==============================] - 0s 20ms/step
+++++++++++ [1_1] | 13 ++++++++++++++++++
Elapsed time:  0.06348011800014319
Stopped at epoch:  0
1/1 [==============================] - 0s 23ms/step
+++++++++++ [1_1] | 14 ++++++++++++++++++
Elapsed time:  0.05077494299985119
Stopped at epoch:  0
1/1 [==============================] - 0s 22ms/step
+++++++++++ [1_1] | 15 ++++++++++++++++++
Elapsed time:  0.0428702730000623
Stopped at epoch:  0
1/1 [==============================] - 0s 20ms/st

2024-03-24 20:53:54.500956: W tensorflow/core/data/root_dataset.cc:342] Optimization loop failed: CANCELLED: Operation was cancelled


Elapsed time:  0.79886167199993
Stopped at epoch:  9
1/1 [==============================] - 0s 25ms/step
+++++++++++ [1_4] | 15 ++++++++++++++++++
Elapsed time:  0.6045129249998809
Stopped at epoch:  6
1/1 [==============================] - 0s 23ms/step
+++++++++++ [1_4] | 16 ++++++++++++++++++
Elapsed time:  1.501685877
Stopped at epoch:  18
1/1 [==============================] - 0s 28ms/step
+++++++++++ [1_4] | 17 ++++++++++++++++++
Elapsed time:  2.052123573000017
Stopped at epoch:  28
1/1 [==============================] - 0s 22ms/step
+++++++++++ [1_4] | 18 ++++++++++++++++++
Elapsed time:  0.9465735030000815
Stopped at epoch:  8
1/1 [==============================] - 0s 20ms/step
+++++++++++ [1_4] | 19 ++++++++++++++++++
Elapsed time:  1.0376684679999926
Stopped at epoch:  13
1/1 [==============================] - 0s 30ms/step
+++++++++++ [1_4] | 20 ++++++++++++++++++
Elapsed time:  0.763935851000042
Stopped at epoch:  6
1/1 [==============================] - 0s 22ms/step
+++++++

...

DataFrame salvo em ../results/metrics_7_9
Testando combinacao8: Hidden Size=[45, 22], regularizer=0.1, learning_rate=0.1
+++++++++++ [8_0] | 1 ++++++++++++++++++
Elapsed time:  1.8649602569998933
Stopped at epoch:  32
1/1 [==============================] - 0s 22ms/step
+++++++++++ [8_0] | 2 ++++++++++++++++++
Elapsed time:  0.35221964099991965
Stopped at epoch:  6
1/1 [==============================] - 0s 17ms/step
+++++++++++ [8_0] | 3 ++++++++++++++++++
Elapsed time:  0.34601088699992033
Stopped at epoch:  6
1/1 [==============================] - 0s 27ms/step
+++++++++++ [8_0] | 4 ++++++++++++++++++
Elapsed time:  0.3274611360002382
Stopped at epoch:  6
1/1 [==============================] - 0s 24ms/step
+++++++++++ [8_0] | 5 ++++++++++++++++++
Elapsed time:  0.3058100270000068
Stopped at epoch:  6
1/1 [==============================] - 0s 23ms/step
+++++++++++ [8_0] | 6 ++++++++++++++++++
Elapsed time:  0.3339915800002018
Stopped at epoch:  6
1/1 [==============================] - 0

2024-03-24 21:20:18.025094: W tensorflow/core/data/root_dataset.cc:342] Optimization loop failed: CANCELLED: Operation was cancelled


Elapsed time:  0.4248345679998238
Stopped at epoch:  6
1/1 [==============================] - 0s 27ms/step
+++++++++++ [8_1] | 5 ++++++++++++++++++
Elapsed time:  0.3391769199997725
Stopped at epoch:  6
1/1 [==============================] - 0s 20ms/step
+++++++++++ [8_1] | 6 ++++++++++++++++++
Elapsed time:  0.36899176299993997
Stopped at epoch:  6
1/1 [==============================] - 0s 34ms/step
+++++++++++ [8_1] | 7 ++++++++++++++++++
Elapsed time:  0.39388945199971204
Stopped at epoch:  6
1/1 [==============================] - 0s 21ms/step
+++++++++++ [8_1] | 8 ++++++++++++++++++
Elapsed time:  0.37856033500020203
Stopped at epoch:  6
1/1 [==============================] - 0s 24ms/step
+++++++++++ [8_1] | 9 ++++++++++++++++++
Elapsed time:  0.41481380699997317
Stopped at epoch:  6
1/1 [==============================] - 0s 25ms/step
+++++++++++ [8_1] | 10 ++++++++++++++++++
Elapsed time:  0.3845007719996829
Stopped at epoch:  6
1/1 [==============================] - 0s 22ms/step


,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_8_4_0,0.800740,0.515741,0.861773,0.750718,0.037277,0.069458,0.045829,0.058339
model_8_4_11,0.813250,0.208023,0.686490,0.534012,0.034937,0.113595,0.103944,0.109054
model_8_4_1,0.829789,0.577524,0.868897,0.775514,0.031843,0.060597,0.043467,0.052536
model_8_4_10,0.830703,0.298701,0.718311,0.584649,0.031672,0.100589,0.093394,0.097203
model_8_4_9,0.842573,0.368923,0.739914,0.621837,0.029451,0.090517,0.086231,0.088500
model_8_4_2,0.852502,0.642116,0.865044,0.793903,0.027594,0.051332,0.044745,0.048232
model_8_4_8,0.857081,0.448603,0.767829,0.666301,0.026737,0.079088,0.076976,0.078094
model_8_4_7,0.864205,0.498056,0.785084,0.693851,0.025404,0.071995,0.071256,0.071647
model_8_4_4,0.867016,0.605296,0.797137,0.736683,0.024878,0.056613,0.067259,0.061623
model_8_4_6,0.867699,0.537038,0.794785,0.712967,0.024750,0.066404,0.068039,0.067173


DataFrame salvo em ../results/metrics_8_4
+++++++++++ [8_5] | 1 ++++++++++++++++++
Elapsed time:  0.18148587900031998
Stopped at epoch:  6
1/1 [==============================] - 0s 16ms/step
+++++++++++ [8_5] | 2 ++++++++++++++++++
Elapsed time:  0.16980518999980632
Stopped at epoch:  6
1/1 [==============================] - 0s 16ms/step
+++++++++++ [8_5] | 3 ++++++++++++++++++
Elapsed time:  0.17415179699992223
Stopped at epoch:  6
1/1 [==============================] - 0s 16ms/step
+++++++++++ [8_5] | 4 ++++++++++++++++++
Elapsed time:  0.17023862199994255
Stopped at epoch:  6
1/1 [==============================] - 0s 16ms/step
+++++++++++ [8_5] | 5 ++++++++++++++++++
Elapsed time:  0.22453268600020237
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [8_5] | 6 ++++++++++++++++++
Elapsed time:  0.16993519800007562
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [8_5] | 7 ++++++++++++++++++
Elapsed time:  0.199525